In [19]:
import pandas as pd
import numpy as np
import os
import statsmodels.tsa.arima_model as smt
#from pandas.core import datetools

In [20]:
df_stocks = pd.read_csv('top25_stocks3.csv', index_col = 0)
df_stocks = df_stocks[df_stocks['TimeFrame'] == '2w'].reset_index()
stocks = df_stocks['stock_name']
stocks

0     CANBK
1       DLF
2    RECLTD
Name: stock_name, dtype: object

In [21]:
df_ADANI = pd.read_csv("./stock_data/" + 'ADANIENT' +".csv", names = ('Dates', 'stockVWAP', 'futureVWAP', 'bidPrice', 'askPrice', 'total_value', 'total_size'))
df_ADANI = df_ADANI.iloc[0:60000]
df_ADANI = df_ADANI[df_ADANI.index%60 == 0]

In [22]:
predictions = pd.DataFrame(index = df_ADANI['Dates'], columns = stocks.values)
predictions

,CANBK,DLF,RECLTD
Dates,,,
"('2017-01-02 09:16:00',)",NaN,NaN,NaN
"('2017-01-02 10:16:00',)",NaN,NaN,NaN
"('2017-01-02 11:16:00',)",NaN,NaN,NaN
"('2017-01-02 12:16:00',)",NaN,NaN,NaN
"('2017-01-02 13:16:00',)",NaN,NaN,NaN
...,...,...,...
"('2017-09-04 10:31:00',)",NaN,NaN,NaN
"('2017-09-04 11:31:00',)",NaN,NaN,NaN
"('2017-09-04 12:31:00',)",NaN,NaN,NaN


In [23]:
temp = 0
predictions_all = None
check_empty = 1

In [24]:
count=0
for i in stocks:    
    df = pd.read_csv("./stock_data/" + i +".csv", names = ('Dates', 'stockVWAP', 'futureVWAP', 'bidPrice', 'askPrice', 'total_value', 'total_size'))
    print(df)
    rng = pd.date_range('2017-01-02 09:16:00', periods=1000, freq='60min')
    df = df.iloc[0:60000]
    df = df[df.index%60 == 0]
    df_new = pd.DataFrame(df['stockVWAP'].values, index = rng)

    for k in range(950,999):
        df_scan = df_new.iloc[:k]
        best_aic = np.inf
        best_order = None
        best_mdl = None
        pq_rng = range(4) # [0,1,2,3]
        d_rng = range(2) # [0,1]                        
        for i in pq_rng :
            for d in d_rng :
                for j in pq_rng :  
                    try :
                        tmp_mdl = smt.ARIMA(df_scan, order=(i,d,j)).fit(trend='nc', disp = 0)
                        tmp_aic = tmp_mdl.aic
                        if tmp_aic < best_aic :
                            best_aic = tmp_aic
                            best_order = (i, d, j)
                            best_mdl = tmp_mdl
                    except :
                        continue                                                             
        if best_mdl is None:                                 
            temp = temp + 1                         
        else:
            predict = best_mdl.forecast(steps = 262, alpha = 0.01)[0]
            mean = np.mean(predict)
            threshold = np.std(predict)
            if(df_scan.iloc[k - 1][0] > mean + threshold):
                predictions.iloc[k,count] = 0
            elif(df_scan.iloc[k - 1][0] < mean - threshold):
                predictions.iloc[k,count] = 1 
            else:
                predictions.iloc[k,count] = 0.5
    count=count+1
predictions_new = pd.DataFrame(predictions)
predictions_new.to_csv("predictions_2w(14d).csv")
print("predictions_2w(14d).csv" + " Ready!!!")

                          Dates   stockVWAP  futureVWAP  bidPrice  askPrice  \
0      ('2017-01-02 09:16:00',)  265.238848  263.438235    265.30    265.60   
1      ('2017-01-02 09:17:00',)  264.777081  263.302778    264.65    265.05   
2      ('2017-01-02 09:18:00',)  264.573172  262.932353    264.60    265.00   
3      ('2017-01-02 09:19:00',)  264.766859  262.930769    264.60    264.95   
4      ('2017-01-02 09:20:00',)  264.173364  261.661538    264.15    264.60   
...                         ...         ...         ...       ...       ...   
89995  ('2017-12-29 15:26:00',)  360.266931  362.533333    360.20    360.50   
89996  ('2017-12-29 15:27:00',)  360.536784  362.577778    360.50    360.90   
89997  ('2017-12-29 15:28:00',)  360.473628  362.590000    360.50    360.75   
89998  ('2017-12-29 15:29:00',)  360.413091  362.592857    360.30    360.70   
89999  ('2017-12-29 15:30:00',)  360.527688  362.878846    360.20    361.30   

       total_value    total_size  
0          25412

C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', Hes

C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals

                          Dates   stockVWAP  futureVWAP  bidPrice  askPrice  \
0      ('2017-01-02 09:16:00',)  115.625189  115.473077    115.55    115.90   
1      ('2017-01-02 09:17:00',)  116.060935  116.079592    115.95    116.25   
2      ('2017-01-02 09:18:00',)  116.067631  116.150000    116.00    116.25   
3      ('2017-01-02 09:19:00',)  115.698767  115.744286    115.65    116.00   
4      ('2017-01-02 09:20:00',)  115.470345  115.529167    115.40    115.65   
...                         ...         ...         ...       ...       ...   
89995  ('2017-12-29 15:26:00',)  259.289994  261.104000    259.30    259.55   
89996  ('2017-12-29 15:27:00',)  259.470885  261.306757    259.40    259.75   
89997  ('2017-12-29 15:28:00',)  259.792539  261.655435    259.80    260.35   
89998  ('2017-12-29 15:29:00',)  259.701901  261.658333    259.80    260.05   
89999  ('2017-12-29 15:30:00',)  260.119944  261.844444    260.00    260.25   

       total_value    total_size  
0         463370

C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals

...

                          Dates   stockVWAP  futureVWAP  bidPrice  askPrice  \
0      ('2017-01-02 09:16:00',)  124.947093  122.050000    124.90    125.35   
1      ('2017-01-02 09:17:00',)  124.523490  121.887500    124.60    124.90   
2      ('2017-01-02 09:18:00',)  124.178644  121.630769    124.15    124.55   
3      ('2017-01-02 09:19:00',)  124.602377  121.830769    124.55    124.85   
4      ('2017-01-02 09:20:00',)  124.703618  121.850000    124.50    124.85   
...                         ...         ...         ...       ...       ...   
89995  ('2017-12-29 15:26:00',)  155.746567  156.585484    155.70    155.95   
89996  ('2017-12-29 15:27:00',)  155.656109  156.632927    155.75    156.00   
89997  ('2017-12-29 15:28:00',)  155.811057  156.697561    155.60    155.90   
89998  ('2017-12-29 15:29:00',)  155.747555  156.557813    155.70    155.95   
89999  ('2017-12-29 15:30:00',)  155.816349  156.577778    155.85    156.10   

       total_value    total_size  
0          11397

C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'avai

C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check m

predictions_2w(14d).csv Ready!!!


C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\Chaitanya\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [18]:
predictions_new

,CANBK,DLF,RECLTD
Dates,,,
"('2017-01-02 09:16:00',)",training,training,training
"('2017-01-02 10:16:00',)",training,training,training
"('2017-01-02 11:16:00',)",training,training,training
"('2017-01-02 12:16:00',)",training,training,training
"('2017-01-02 13:16:00',)",training,training,training
...,...,...,...
"('2017-09-04 10:31:00',)",0,0,0
"('2017-09-04 11:31:00',)",0,0,0
"('2017-09-04 12:31:00',)",0,0,0
